In [1]:
# Importing required libraries
import pandas as pd
import ast
import re

from helper_functions.helpers import clean, structure_conversation_spacy, structure_conversation, Parse_spacy, Parse
from resolution_helpers.object_phrase_resolution import extract_head_verb_object
from resolution_helpers.append_resolved_phrase import append_resolved_vp
from resolution_helpers.append_missing_aux import find_missing_aux_verb
from detect_and_resolve_vpe import detect_resolve_vpe_conversation, detect_resolve_vpe_conversation_spacy
from compare_with_annotations import compare_head_verb, compare_ellided_obj_parent

import json

# Reading Annotated File
annotated_df = pd.read_excel("./Data/test.xlsx")

# Currently Checking Accuracy on Test/Train
text = annotated_df['english'].tolist()
annotations = annotated_df[annotated_df.columns[3]].tolist()
obj_only_ellipsis_annotations = annotated_df[annotated_df.columns[4]].tolist()

# Removing empty lines from the list
for line in text:
    if(line == '' or line == '\n'):
        text.remove(line)

# Counting lines and number of cases where ellipsis is detected and where it isn't detected
cases_ellipsis = 0
cases_no_ellipsis = 0

error_conversations = []

detect_resolve_output_file = open("./Outputs/detect_resolve_output_annotated.txt", "w+")

corr_count = 0
ann_count = 0

conversation_final_vpe_list = []

2023-04-30 19:14:26.746188: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-04-30 19:14:26.746277: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining 

In [2]:
# Running it for all sentences in our list
for index, conversation in enumerate(text):
   
    print(str(index) + ") ", end = " ", file = detect_resolve_output_file)
    print(conversation, end = "\n\n", file = detect_resolve_output_file)

    temp_entry_output_storage_file = open("./Outputs/TempConversationalOutput/test_entry_" + str(index) + ".txt", "w")

    print("Entry - %d\n" %(index), file = temp_entry_output_storage_file)
    print("Scoring in STANFORD\n", file = temp_entry_output_storage_file)
    print("Scoring in STANFORD\n", file = detect_resolve_output_file)
    vpe_conversation_stanford = detect_resolve_vpe_conversation(clean(conversation), detect_resolve_output_file, temp_entry_output_storage_file)
    
    print("Scoring in SPACY\n", file = temp_entry_output_storage_file)
    print("Scoring in SPACY\n", file = detect_resolve_output_file)
    vpe_conversation_spacy = detect_resolve_vpe_conversation_spacy(clean(conversation), detect_resolve_output_file, temp_entry_output_storage_file)
    
    detected_vpe_cases_stanford = []
    for output_arr in vpe_conversation_stanford:
        if output_arr != []:
            detected_vpe_cases_stanford.append(output_arr)
            
    detected_vpe_cases_spacy = []
    for output_arr in vpe_conversation_spacy:
        if output_arr != []:
            detected_vpe_cases_spacy.append(output_arr)
    
    # Taking only those detections where both the parsers detected VPE in the conversation
    final_vpe_list = []
    
    for vpe_dict_stanford in detected_vpe_cases_stanford:
        for vpe_dict_spacy in detected_vpe_cases_spacy:
            if vpe_dict_stanford['category'] == vpe_dict_spacy['category'] and vpe_dict_stanford['site'] == vpe_dict_spacy['site'] and vpe_dict_stanford['sen_index'] == vpe_dict_spacy['sen_index']:
                
                # If Main verb is resolved and both Stanford Spacy head verb resolutions match then choose any since they both are the same
                if vpe_dict_stanford['resolved_head_verb'] is not None and vpe_dict_spacy['resolved_head_verb'] is not None and vpe_dict_stanford['resolved_head_verb'] == vpe_dict_spacy['resolved_head_verb']:
                    final_vpe_list.append(vpe_dict_stanford)
                # If Main verb is resolved and Stanford Spacy head verb resolutions don't match then giving preference to Spacy
                if vpe_dict_stanford['resolved_head_verb'] is not None and vpe_dict_spacy['resolved_head_verb'] is not None and vpe_dict_stanford['resolved_head_verb'] != vpe_dict_spacy['resolved_head_verb']:
                    final_vpe_list.append(vpe_dict_spacy)
                # If Main verb is resolved for Spacy but not for Stanford then choose any since they Spacy
                if vpe_dict_stanford['resolved_head_verb'] is None and vpe_dict_spacy['resolved_head_verb'] is not None:
                    final_vpe_list.append(vpe_dict_spacy)
                # If Main verb is resolved for Stanford but not for Spacy then choose any since they Stanford
                if vpe_dict_stanford['resolved_head_verb'] is not None and vpe_dict_spacy['resolved_head_verb'] is None:
                    final_vpe_list.append(vpe_dict_stanford)
                # If Main verb is not resolved for neither of them it implies Only the Object child is ellided and if the Ellided Object child's parents matches then choose any since they are the same. 
                # Note that matched Ellided Object Child's parent could also be None which would imply we couldn't resolve the ellipsis successfully
                if vpe_dict_stanford['resolved_head_verb'] is None and vpe_dict_spacy['resolved_head_verb'] is None and vpe_dict_stanford['ellided_obj_child_parent_verb'] == vpe_dict_spacy['ellided_obj_child_parent_verb']:
                    final_vpe_list.append(vpe_dict_spacy)
                # If Main verb is not resolved for neither of them it implies Only the Object child is ellided and if the Ellided Object child's parents don't match then choose Spacy
                if vpe_dict_stanford['resolved_head_verb'] is None and vpe_dict_spacy['resolved_head_verb'] is None and vpe_dict_stanford['ellided_obj_child_parent_verb'] != vpe_dict_spacy['ellided_obj_child_parent_verb']:
                    final_vpe_list.append(vpe_dict_spacy)
                

    # To check category wise performance select category from 1-5 or keep it None
    comparison_category = None
    
    structured_conversation = structure_conversation_spacy(clean(conversation))[0]

    corr_count, ann_count, print_flag_pure = compare_head_verb(final_vpe_list, annotations[index], corr_count, ann_count, comparison_category, temp_entry_output_storage_file)
    corr_count, ann_count, print_flag_obj_only = compare_ellided_obj_parent(structured_conversation, final_vpe_list, obj_only_ellipsis_annotations[index], corr_count, ann_count, comparison_category, temp_entry_output_storage_file)
    
    final_vpe_list = extract_head_verb_object(final_vpe_list, structure_conversation_spacy(clean(conversation)))
    final_vpe_list = find_missing_aux_verb(final_vpe_list, structure_conversation_spacy(clean(conversation)))
    
    conversation_final_vpe_list.append(final_vpe_list)
    
    print("\nFINAL VPE\n", file = temp_entry_output_storage_file)
    print(final_vpe_list, file = temp_entry_output_storage_file)
    print("\nFINAL VPE\n", file = detect_resolve_output_file)
    print(final_vpe_list, file = detect_resolve_output_file)
    
    print("\nACTUAL ANNOTATIONS\n", file = temp_entry_output_storage_file)
    print(annotations[index], file = temp_entry_output_storage_file)
    print("\nACTUAL ANNOTATIONS\n", file = detect_resolve_output_file)
    print(annotations[index], file = detect_resolve_output_file)
    
    if ann_count>0:
        print("\nCURRENT ACCURACY ==> %d/%d = %f\n" % (corr_count, ann_count, corr_count/ann_count), file = temp_entry_output_storage_file)
    
    print("================================= \n\n", file = temp_entry_output_storage_file)
    
    print("================================= \n\n", file = detect_resolve_output_file)
    
    temp_entry_output_storage_file.close()
    
#     if print_flag_pure or print_flag_obj_only:
    temp_entry_read_file = open("./Outputs/TempConversationalOutput/test_entry_" + str(index) + ".txt", "r")
    erroneous_entry = temp_entry_read_file.read()
    print(erroneous_entry)
    temp_entry_read_file.close()
    
detect_resolve_output_file.close()

Entry - 0

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[0]
{2: 0, 5: 0}
[{3: 0}]


AFTER VERB LIST FILTERING
[0]
{}
[{}]


AFTER NOMINAL SUBJECT BASED SCORING
[0]
{}
[{}]


AFTER COMP CLAUSE BASED SCORING
[0]
{}
[{}]


AFTER AUX BASED SCORING
[0]
{}
[{}]


AFTER BACKTRACKING BASED SCORING
[0]
{}
[{}]


BEFORE VERB LIST FILTERING
[7, 8, 9]
{3: 0}
[{}, {1: 0, 3: 0, 4: 0, 5: 0, 13: 0}, {0: 0, 2: 0}]


AFTER VERB LIST FILTERING
[7, 8, 9]
{}
[{}, {1: 0, 3: 0, 5: 0, 13: 0}, {2: 0}]


Prev 8 - 1 NUMBER + 1
Prev 8 - 5 SAME SUBJECT FP SP NO SWITCH + 2
Prev 8 - 13 SAME SUBJECT FP SP NO SWITCH + 2
Prev 9 - 2 NUMBER + 1
Prev 9 - 2 FP SP SWITCH + 1
AFTER NOMINAL SUBJECT BASED SCORING
[7, 8, 9]
{}
[{}, {1: 1, 3: 0, 5: 2, 13: 2}, {2: 2}]


AFTER COMP CLAUSE BASED SCORING
[7, 8, 9]
{}
[{}, {1: 1, 3: 0, 5: 2, 13: 2}, {2: 2}]


AFTER AUX BASED SCORING
[7, 8, 9]
{}
[{}, {1: 1, 3: 0, 5: 2, 13: 2}, {2: 3}]


AFTER BACKTRACKING BASED SCORING
[7, 8, 9]
{}
[{}, {1: 1, 3: 0, 5: 2, 13: 2}, {2: 3}]


AFTER A

Entry - 2

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[15, 16, 17]
{}
[{2: 0}, {4: 0}, {1: 0, 5: 0}]


AFTER VERB LIST FILTERING
[15, 16, 17]
{}
[{2: 0}, {4: 0}, {1: 0, 5: 0}]


Prev 15 - 2 NUMBER + 1
Prev 16 - 4 NUMBER + 1
Prev 17 - 1 NUMBER + 1
Prev 17 - 5 NUMBER + 1
Prev 17 - 5 TP + 1
AFTER NOMINAL SUBJECT BASED SCORING
[15, 16, 17]
{}
[{2: 1}, {4: 1}, {1: 1, 5: 2}]


AFTER COMP CLAUSE BASED SCORING
[15, 16, 17]
{}
[{2: 1}, {4: 1}, {1: 1, 5: 2}]


AFTER AUX BASED SCORING
[15, 16, 17]
{}
[{2: 2}, {4: 1}, {1: 1, 5: 3}]


AFTER BACKTRACKING BASED SCORING
[15, 16, 17]
{}
[{2: 2}, {4: 1}, {1: 1, 5: 3}]


AFTER ADDING SCORE MULTIPLIERS
[15, 16, 17]
{}
[{2: 1.3}, {4: 0.8}, {1: 1, 5: 3}]
-----------------------------


Scoring in SPACY

BEFORE VERB LIST FILTERING
[15, 16, 17]
{}
[{2: 0}, {4: 0}, {1: 0, 5: 0}]


AFTER VERB LIST FILTERING
[15, 16, 17]
{}
[{2: 0}, {4: 0}, {1: 0, 5: 0}]


Prev 15 - 2 NUMBER + 1
Prev 16 - 4 NUMBER + 1
Prev 17 - 1 NUMBER + 1
Prev 17 - 5 NUMBER + 1
Prev 17 - 

Entry - 5

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[7, 8, 9]
{2: 0, 6: 0}
[{1: 0, 4: 0}, {}, {1: 0, 2: 0, 4: 0}]


AFTER VERB LIST FILTERING
[7, 8, 9]
{2: 0}
[{4: 0}, {}, {1: 0, 2: 0, 4: 0}]


2 SAME SUBJECT EXACT MATCH + 2
Prev 7 - 4 SAME SUBJECT TP + 2
Prev 9 - 2 NUMBER + 1
Prev 9 - 4 NUMBER + 1
AFTER NOMINAL SUBJECT BASED SCORING
[7, 8, 9]
{2: 2}
[{4: 2}, {}, {1: 0, 2: 1, 4: 1}]


AFTER COMP CLAUSE BASED SCORING
[7, 8, 9]
{2: 2}
[{4: 2}, {}, {1: 0, 2: 1, 4: 0}]


AFTER AUX BASED SCORING
[7, 8, 9]
{2: 2}
[{4: 3}, {}, {1: 0, 2: 1, 4: 0}]


AFTER BACKTRACKING BASED SCORING
[7, 8, 9]
{2: 2}
[{4: 3}, {}, {1: 0, 2: 1, 4: 0}]


AFTER ADDING SCORE MULTIPLIERS
[7, 8, 9]
{2: 2}
[{4: 1.9500000000000002}, {}, {1: 0, 2: 1, 4: 0}]
-----------------------------


Scoring in SPACY

BEFORE VERB LIST FILTERING
[7, 8, 9]
{2: 0, 6: 0}
[{1: 0, 4: 0}, {}, {1: 0, 3: 0}]


AFTER VERB LIST FILTERING
[7, 8, 9]
{2: 0}
[{4: 0}, {}, {1: 0, 3: 0}]


2 SAME SUBJECT EXACT MATCH + 2
Prev 7 - 4 SAME SUBJECT 

Entry - 9

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[8, 9, 10]
{1: 0, 3: 0}
[{4: 0, 6: 0}, {5: 0, 7: 0}, {3: 0, 6: 0}]


AFTER VERB LIST FILTERING
[8, 9, 10]
{}
[{}, {}, {}]


AFTER NOMINAL SUBJECT BASED SCORING
[8, 9, 10]
{}
[{}, {}, {}]


AFTER COMP CLAUSE BASED SCORING
[8, 9, 10]
{}
[{}, {}, {}]


AFTER AUX BASED SCORING
[8, 9, 10]
{}
[{}, {}, {}]


AFTER BACKTRACKING BASED SCORING
[8, 9, 10]
{}
[{}, {}, {}]


AFTER ADDING SCORE MULTIPLIERS
[8, 9, 10]
{}
[{}, {}, {}]
-----------------------------


BEFORE VERB LIST FILTERING
[8, 9, 10]
{1: 0, 3: 0}
[{4: 0, 6: 0}, {5: 0, 7: 0}, {3: 0, 6: 0}]


AFTER VERB LIST FILTERING
[8, 9, 10]
{}
[{}, {}, {}]


AFTER NOMINAL SUBJECT BASED SCORING
[8, 9, 10]
{}
[{}, {}, {}]


AFTER COMP CLAUSE BASED SCORING
[8, 9, 10]
{}
[{}, {}, {}]


AFTER AUX BASED SCORING
[8, 9, 10]
{}
[{}, {}, {}]


AFTER BACKTRACKING BASED SCORING
[8, 9, 10]
{}
[{}, {}, {}]


Scoring in SPACY

BEFORE VERB LIST FILTERING
[8, 9, 10]
{1: 0, 3: 0}
[{4: 0, 6: 0}, {5: 0, 7: 

Entry - 13

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[2, 3, 4]
{3: 0}
[{2: 0}, {}, {0: 0}]


AFTER VERB LIST FILTERING
[2, 3, 4]
{}
[{}, {}, {}]


AFTER NOMINAL SUBJECT BASED SCORING
[2, 3, 4]
{}
[{}, {}, {}]


AFTER COMP CLAUSE BASED SCORING
[2, 3, 4]
{}
[{}, {}, {}]


AFTER AUX BASED SCORING
[2, 3, 4]
{}
[{}, {}, {}]


AFTER BACKTRACKING BASED SCORING
[2, 3, 4]
{}
[{}, {}, {}]


AFTER ADDING SCORE MULTIPLIERS
[2, 3, 4]
{}
[{}, {}, {}]
-----------------------------


BEFORE VERB LIST FILTERING
[4, 5, 6]
{3: 0}
[{0: 0}, {3: 0}, {0: 0}]


AFTER VERB LIST FILTERING
[4, 5, 6]
{}
[{}, {}, {}]


AFTER NOMINAL SUBJECT BASED SCORING
[4, 5, 6]
{}
[{}, {}, {}]


AFTER COMP CLAUSE BASED SCORING
[4, 5, 6]
{}
[{}, {}, {}]


AFTER AUX BASED SCORING
[4, 5, 6]
{}
[{}, {}, {}]


AFTER BACKTRACKING BASED SCORING
[4, 5, 6]
{}
[{}, {}, {}]


AFTER ADDING SCORE MULTIPLIERS
[4, 5, 6]
{}
[{}, {}, {}]
-----------------------------


BEFORE VERB LIST FILTERING
[8, 9, 10]
{3: 0}
[{2: 0}, {1: 0}, {0: 0}]

...

Entry - 381

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[0]
{1: 0}
[{1: 0, 3: 0, 6: 0}]


AFTER VERB LIST FILTERING
[0]
{}
[{1: 0, 3: 0}]


Prev 0 - 3 NUMBER + 1
Prev 0 - 3 FP SP SWITCH + 1
AFTER NOMINAL SUBJECT BASED SCORING
[0]
{}
[{1: 0, 3: 2}]


AFTER COMP CLAUSE BASED SCORING
[0]
{}
[{1: 0, 3: 1}]


AFTER AUX BASED SCORING
[0]
{}
[{1: 0, 3: 1}]


AFTER BACKTRACKING BASED SCORING
[0]
{}
[{1: 0, 3: 1}]


AFTER ADDING SCORE MULTIPLIERS
[0]
{}
[{1: 0, 3: 1}]
-----------------------------


Scoring in SPACY

BEFORE VERB LIST FILTERING
[0]
{1: 0}
[{1: 0, 3: 0, 6: 0}]


AFTER VERB LIST FILTERING
[0]
{}
[{1: 0, 3: 0}]


Prev 0 - 3 NUMBER + 1
Prev 0 - 3 FP SP SWITCH + 1
AFTER NOMINAL SUBJECT BASED SCORING
[0]
{}
[{1: 0, 3: 2}]


AFTER COMP CLAUSE BASED SCORING
[0]
{}
[{1: 0, 3: 1}]


AFTER AUX BASED SCORING
[0]
{}
[{1: 0, 3: 1}]


AFTER BACKTRACKING BASED SCORING
[0]
{}
[{1: 0, 3: 1}]


AFTER ADDING SCORE MULTIPLIERS
[0]
{}
[{1: 0, 3: 1}]
-----------------------------



COMPARING PUR

Entry - 385

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[0, 1, 2]
{1: 0, 8: 0}
[{}, {1: 0}, {1: 0}]


AFTER VERB LIST FILTERING
[0, 1, 2]
{}
[{}, {}, {}]


AFTER NOMINAL SUBJECT BASED SCORING
[0, 1, 2]
{}
[{}, {}, {}]


AFTER COMP CLAUSE BASED SCORING
[0, 1, 2]
{}
[{}, {}, {}]


AFTER AUX BASED SCORING
[0, 1, 2]
{}
[{}, {}, {}]


AFTER BACKTRACKING BASED SCORING
[0, 1, 2]
{}
[{}, {}, {}]


Scoring in SPACY

BEFORE VERB LIST FILTERING
[0, 1, 2]
{1: 0, 8: 0}
[{}, {1: 0}, {1: 0}]


AFTER VERB LIST FILTERING
[0, 1, 2]
{}
[{}, {}, {}]


AFTER NOMINAL SUBJECT BASED SCORING
[0, 1, 2]
{}
[{}, {}, {}]


AFTER COMP CLAUSE BASED SCORING
[0, 1, 2]
{}
[{}, {}, {}]


AFTER AUX BASED SCORING
[0, 1, 2]
{}
[{}, {}, {}]


AFTER BACKTRACKING BASED SCORING
[0, 1, 2]
{}
[{}, {}, {}]



COMPARING OBJECT ONLY ELLIPSIS RESOLUTIONS WITH ITS ANNOTATIONS

[]
[]

0 resolutions out of 0 are correct.

FINAL VPE

[{'category': 1, 'site': 8, 'licensor': 'is', 'lemmatized_licensor': 'be', 'sen_index': 3, 'resolve

Entry - 390

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[5, 6, 7]
{3: 0}
[{1: 0, 6: 0}, {1: 0, 3: 0, 5: 0}, {}]


AFTER VERB LIST FILTERING
[5, 6, 7]
{}
[{}, {}, {}]


AFTER NOMINAL SUBJECT BASED SCORING
[5, 6, 7]
{}
[{}, {}, {}]


AFTER COMP CLAUSE BASED SCORING
[5, 6, 7]
{}
[{}, {}, {}]


AFTER AUX BASED SCORING
[5, 6, 7]
{}
[{}, {}, {}]


AFTER BACKTRACKING BASED SCORING
[5, 6, 7]
{}
[{}, {}, {}]


AFTER ADDING SCORE MULTIPLIERS
[5, 6, 7]
{}
[{}, {}, {}]
-----------------------------


BEFORE VERB LIST FILTERING
[10, 11, 12]
{1: 0}
[{1: 0, 2: 0}, {1: 0, 5: 0}, {1: 0}]


AFTER VERB LIST FILTERING
[10, 11, 12]
{}
[{2: 0}, {5: 0}, {}]


Prev 10 - 2 SAME SUBJECT FP SP NO SWITCH + 2
Prev 11 - 5 NUMBER + 1
Prev 11 - 5 FP SP SWITCH + 1
AFTER NOMINAL SUBJECT BASED SCORING
[10, 11, 12]
{}
[{2: 2}, {5: 2}, {}]


AFTER COMP CLAUSE BASED SCORING
[10, 11, 12]
{}
[{2: 2}, {5: 2}, {}]


AFTER AUX BASED SCORING
[10, 11, 12]
{}
[{2: 2}, {5: 3}, {}]


AFTER BACKTRACKING BASED SCORING
[10, 11, 12

Entry - 394

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[3, 4, 5]
{}
[{}, {1: 0, 3: 0}, {}]


AFTER VERB LIST FILTERING
[3, 4, 5]
{}
[{}, {1: 0, 3: 0}, {}]


Prev 4 - 1 IMPERATIVE SUBJECT AND MODAL ELLIPSIS RESPONSE + 1
Prev 4 - 3 SAME SUBJECT FP SP NO SWITCH + 2
AFTER NOMINAL SUBJECT BASED SCORING
[3, 4, 5]
{}
[{}, {1: 1, 3: 2}, {}]


AFTER COMP CLAUSE BASED SCORING
[3, 4, 5]
{}
[{}, {1: 1, 3: 2}, {}]


AFTER AUX BASED SCORING
[3, 4, 5]
{}
[{}, {1: 1, 3: 2}, {}]


AFTER BACKTRACKING BASED SCORING
[3, 4, 5]
{}
[{}, {1: 1, 3: 2}, {}]


AFTER ADDING SCORE MULTIPLIERS
[3, 4, 5]
{}
[{}, {1: 0.8, 3: 1.6}, {}]
-----------------------------


BEFORE VERB LIST FILTERING
[7, 8, 9]
{1: 0, 3: 0, 6: 0}
[{3: 0}, {2: 0}, {0: 0, 5: 0}]


AFTER VERB LIST FILTERING
[7, 8, 9]
{3: 0}
[{3: 0}, {2: 0}, {}]


3 NUMBER + 1
Prev 8 - 2 SAME SUBJECT FP SP NO SWITCH + 2
AFTER NOMINAL SUBJECT BASED SCORING
[7, 8, 9]
{3: 1}
[{3: 0}, {2: 2}, {}]


AFTER COMP CLAUSE BASED SCORING
[7, 8, 9]
{3: 1}
[{3: 0}, {2: 2

Entry - 396

Scoring in STANFORD

BEFORE VERB LIST FILTERING
[11, 12, 13]
{2: 0, 8: 0}
[{3: 0, 10: 0, 14: 0, 17: 0, 19: 0, 23: 0}, {}, {0: 0, 2: 0}]


AFTER VERB LIST FILTERING
[11, 12, 13]
{}
[{10: 0, 17: 0, 19: 0}, {}, {2: 0}]


Prev 13 - 2 NUMBER + 1
Prev 13 - 2 TP - 1
AFTER NOMINAL SUBJECT BASED SCORING
[11, 12, 13]
{}
[{10: 0, 17: 0, 19: 0}, {}, {2: 0}]


AFTER COMP CLAUSE BASED SCORING
[11, 12, 13]
{}
[{10: 0, 17: 0, 19: 0}, {}, {2: 0}]


AFTER AUX BASED SCORING
[11, 12, 13]
{}
[{10: 1, 17: 1, 19: 0}, {}, {2: 0}]


AFTER BACKTRACKING BASED SCORING
[11, 12, 13]
{}
[{10: 1, 17: 1, 19: 0}, {}, {2: 0}]


AFTER ADDING SCORE MULTIPLIERS
[11, 12, 13]
{}
[{10: 0.2, 17: 0.2, 19: 0.0}, {}, {2: 0}]
-----------------------------


Scoring in SPACY


COMPARING PURE ELLIPSIS RESOLUTIONS WITH ITS ANNOTATIONS

[['forget', "But , it's probably not a good idea to keep it up I'm willing to forgive and forget and if you are ."]]
[]

0 resolutions out of 1 are correct.

FINAL VPE

[]

ACTUAL ANNOTATI

In [3]:
for index, conversation in enumerate(text):
    print(str(index) + ") ", end = "\n")
    cleaned_text = re.sub("__eou__", "", text[index])
    cleaned_text = re.sub(' +', ' ', cleaned_text)
    cleaned_text = cleaned_text.strip()
    print(cleaned_text, end = "\n\n")
    resolved_conversation, resolved_conversation_parsed = append_resolved_vp(conversation_final_vpe_list[index], clean(text[index]))
    print(resolved_conversation, end = "\n\n=================================\n\n")

0) 
so , what ’ s new in the kitchen ? That refrigerator is new , isn ’ t it ? yes . I needed a large one . Before , I had a separate refrigerator and freezer , but this has both combined into one . That ’ s usual nowadays . You ’ Ve added a few shelves too . yes . You know I ’ Ve been cooking more kind of food recently and I needed some extra space for spices and ingredients . Did you buy new cupboards too ? no , I didn ’ t . I gave them a really good clean , so they just look new . The worktop was in poor condition , so I had a new one added . I see that you have bought several new pots and pans and utensils . yes , I have . I need them to help me with these new dished I ’ m trying to make . I need a little more practice before I invite guests over .

so , what's new in the kitchen ? That refrigerator is new , isn't it ? yes . I needed a large one . Before , I had a separate refrigerator and freezer , but this has both combined into one . That's usual nowadays . You'Ve added a few sh

good morning . I understand that you'Ve got a problem with your washing machine . I'm from the repair company . excellent . Come in please . The washing machine is in the bathroom upstairs . It keeps breaking down . when did it first break down ? about ten days ago . I'Ve tried to use it since then . Sometimes it works and sometimes it doesn't works . it's very frustrating . is it still under warranty . If it is and I can't fix it , it would be quicker and easier to exchange it for a new one . yes , it's still under warranty . Over the last few weeks , it's also been making a high - pitch noise when it's in use . okay . I'll start by looking at the motor . I'll just unplug it and take a look inside the machine ... oh , yes . There's the problem . It's quite simple . I'll sort it out in a few minutes . what's wrong with it ? part of the motor is loose . I can put it back in place quite easily . that's great . Thanks very much . Would you like a cup of tea or coffee ?


6) 
Hey , Jim . I

I would like to talk to you , if you have time . What's going on ? I want to discuss some things about my lease . What aren't you too sure about ? Can you tell me when my lease is up ? Your lease is for three years . If I ever decide to leave before the three years , will there be a problem ? If you do decide , I will keep your deductible . Why is that ? If you break the contract , then you will not receive your deductible . Oh , I see . All right . I'm glad I could clarify things for you .


17) 
What kind of place shall we rent ? It should be close to the university . Neither of us are good at getting up in the mornings and closer it is , the later we can get up . Absolutely . That's the most important thing to take into consideration . I'm not too worried about the size of the flat . Neither am I . So a small place is ok , but we'll get a bigger one if it's not expensive . Do you mind if it's in a noisy area ? I don't mind . i'm not a light sleeper , but quieter is better for when w

do you like animals ? yes , doesn't everyone ? what's your favorite animal ? it's hard to say . While dogs are more affectionate , cats are much easier to take care of . have you ever had a dog ? when I was little , we had a few domestic dogs at home . they always say that dogs are a man's best friends . do you think that's true ? definitely . rich or poor , dogs always love their masters . how about cats ? Have you ever had one for a pet ? i had one a few months ago , but she ended up running away . how did that happen ? she left through a window that was open one night and never returned . you must have been devastated ! i was pretty sad . i still hope that she'll return some day ! I hope she does return . Do you think you'll ever get another pet ? I think I will get another pet . It's difficult to feel longly when you have a pet around . i hope your cat finds its way back home . i do think , too .


24) 
Why don't we get you some shirts ? I want to leave . We've already been here tw

What are you doing , karen ? I'm making a shopping-list , Tom . What do we need ? We need a lot of things this week . I must go to the grocer's . We haven't got much tea or coffee , and we haven't got any sugar or jam . What about vegetables ? I must go to the greengrocer's . We haven't got many tomatoes , but we've got a lot of potatoes . I must go to the butcher's , too . We need some meat . We haven't got any meat at all . Have we got any beer and wine ? No , we haven't got any meat . And I'm not going to get any ! I hope that you've got some money . I haven't got much . Well , I haven't got much either !


32) 
Do you want any meat today , Mrs Bird ? Yes , please . Do you want beef or lamb ? Beef , please . This lamb's very good . I like lamb , but my husband doesn't . What about some steak ? This is a nice piece . Give me that piece please , and a pound of meat , too . Do you want a chicken , Mrs Bird ? They're very nice . No , thank you. My husband likes steak , but he doesn't li

...

You didn't ring me last night . You said you would ring me . I'm sorry to have made you disappointed . That's all right . But why were you so rude to me at lunch . Was I rude to you at lunch ? Sorry , I didn't mean to be . I do apologize . And why are you yarning now ? Are you bored ? Forgive me darling . I'm very tired .


335) 
I wish you would not be so selfish . Selfishness is one human beings instincts . It is very difficult to get it out root and branch . But you should at least consider your son's future . His mother , not me , should .

I wish you would not be so selfish . Selfishness is one human beings instincts . It is very difficult to get it out root and branch . But you should at least consider your son's future . His mother , not me , should consider my son future .


336) 
The trouble is not that . It is that he may suddenly remember something I promised him a couple of weeks ago , out of a clear blue sky . Then he complains that I have gone back on my words . Does he d

Ah ! No ! Damn it ! It's a blackout . Now I can't see Seinfeld . So what ? I just lost one hour's worth of work . Really ? How could you do that ? Don't you save every couple minutes ? No , I didn't save this time.Damn it ! And I'm sick of writing this paper . Now I have to write it all over again too . I've had that problem too many times . So I learned to save.When I'm writing something , I save every three sentences or so . I don't want to lose anything . I hate computers . Sometimes I think they cause more trouble than they're worth . What are we going to do now ? I don't know . I feel like going out . I wonder how much of the city is down . It doesn't matter . I still can go out and buy a beer . Maybe . But if there's a blackout , probably the pubs are closed.And besides , I know you have a political science exam tomorrow . I thought you had a lot of reading to do . Yes , that's true . Damn ! I just lost half my paper , and now I can't even do my homework.This is a bad time for th

Alice , I never knew you had such a lovely voice . You really can sing , can't you ? Thanks , Mark . I used to be a member of the school choir . No wonder you can control your voice so well . You are a professional singer . Well , you are flattering me . I wouldn't say I am a professional , but I did receive some training at school . My music teacher used to be a professional singer . Well , a good teacher makes good students . You are quite right .


358) 
I wish you wouldn't play the TV so loud . Sorry . Were you trying to sleep ? Yes , please ask when you want to borrow my records next time . I'm sorry . You're right . I should have .

I wish you wouldn't play the TV so loud . Sorry . Were you trying to sleep ? Yes , please ask when you want to borrow my records next time . I'm sorry . You're right . I should have borrow your records .


359) 
You don't have to play that record so loud , do you ? I'm sorry . Has it been bothering you ? Yes . And please ask me the next time you want 

okay . Come back into the classroom , class . Does the class start again , Madam ? Yes this break has gone on long enough . Oh , breaktime flies . come on , baby . It's time now . Madam , another minute , could I ? Hurry up , little boy .


370) 
I give up ! I simply can't learn French ! Why do you say that ? I think you're making a lot of progress . No , I'm not . I try and try and I still can't speak it very well . Learning any language takes a lot of effort.But don't give up . Why don't we practise those dialogues together ? Good idea . That might help .

I give up ! I simply can't learn French ! Why do you say that ? I think you're making a lot of progress . No , I 'm not making a lot of progress . I try and try and I still can't speak it very well . Learning any language takes a lot of effort.But don't give up . Why don't we practise those dialogues together ? Good idea . That might help .


371) 
Are you new here ? Yes , I am . Is this your first year in college ? I actually tran

Did you go to the movie yesterday ? Yes , I did go to the movie . What did you think of it ? I don't think much of the film . I think it is the worst one I have ever seen . Oh , I'm not sure I agree with you . It was not a good one , but it was not the worst one . Actually , it wasn't too bad . I can't think the same . What's more , some scenes were good , much fit for the children . I couldn't agree with you .


387) 
You didn't call me last night . You said you would . I'm sorry to have made you disappointed . That's all right . But why were you so rude to me at lunch . Was I rude ? Sorry , I didn't mean to be . I do apologize . And why are you yarning now ? Are you bored ? Forgive me darling . I'm very tired .

You didn't call me last night . You said you would call me . I'm sorry to have made you disappointed . That's all right . But why were you so rude to me at lunch . Was I rude ? Sorry , I didn't mean to be . I do apologize . And why are you yarning now ? Are you bored ? Forgiv